In [139]:
from IPython.display import display
import altair as alt

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

# Récuperer les données

In [146]:
import duckdb

# Le dataset
csv = '../../data/toflit18_all_flows.csv'

# Lire le csv et l'envoyer comme une relation (table)
toflit_flows = duckdb.sql(f"""
SELECT  customs_region,
        customs_office,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart,
        product_revolutionempire
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
""")

toflit_flows.show()

┌────────────────┬────────────────┬───────────┬─────────┬───────────────┬──────────────────────┬───────────────────────┐
│ customs_region │ customs_office │   value   │  year   │ export_import │ best_guess_region_…  │ product_revolutione…  │
│    varchar     │    varchar     │   float   │ varchar │    varchar    │       varchar        │        varchar        │
├────────────────┼────────────────┼───────────┼─────────┼───────────────┼──────────────────────┼───────────────────────┤
│ NULL           │ NULL           │  248210.0 │ 1792    │ Exports       │ 0                    │ Quincaillerie         │
│ NULL           │ NULL           │      NULL │ 1792    │ Exports       │ 0                    │ Alun                  │
│ NULL           │ NULL           │      NULL │ 1792    │ Exports       │ 0                    │ Fruits secs et noix   │
│ NULL           │ NULL           │      NULL │ 1792    │ Exports       │ 0                    │ Fruits secs et noix   │
│ NULL           │ NULL         

In [141]:
# Appliquer les filtres de base (communes à tous)

exports = duckdb.sql(f"""
    FROM toflit_flows
    WHERE value IS NOT NULL
    AND export_import = 'Exports'
    AND best_guess_region_prodxpart = '1'
""")

imports = duckdb.sql(f"""
    FROM toflit_flows
    WHERE value IS NOT NULL
    AND export_import = 'Imports'
    AND best_guess_region_prodxpart = '1'
""")

# Analyser la diversité du marché

In [142]:
def carre_du_marche(export_import, year, bureau_direction, nom):
    
        relation = duckdb.sql(f"""
        FROM {export_import}
        WHERE {bureau_direction} = '{nom}'
        AND year = '{year}'
        """)

        all_product_value = relation.sum('value').fetchone()[0]

        aggreg = duckdb.sql(f"""
        SELECT product_revolutionempire, SUM(value) AS sum_value
        FROM relation
        GROUP BY product_revolutionempire
        """)

        carre = duckdb.sql(f"""
        SELECT  product_revolutionempire,
                sum_value, {all_product_value} AS total,
                power(sum_value/{all_product_value}, 2) AS carre_part_marche
        FROM aggreg
        """)

        return carre.sum('carre_part_marche').fetchone()[0]



# Résultats pour les directions

In [143]:
directions_table = duckdb.query("""
    SELECT DISTINCT customs_region
    FROM toflit_flows
    WHERE customs_region IS NOT NULL
    AND product_revolutionempire IS NOT NULL
""")
directions = [tuple[0] for tuple in directions_table.fetchall()]

In [144]:
carre_du_marche_directions = []
for direction in directions:
    try:
        carre = carre_du_marche(
            export_import='exports',
            year='1789',
            bureau_direction='customs_region',
            nom=direction)
    except:
        print(f"Pas de résultat pour {direction}")
    else:
        carre_du_marche_directions.append({
            'direction': direction,
            'carre_du_marche' : carre
        })

carre_du_marche_directions

Pas de résultat pour France par la Compagnie des Indes
Pas de résultat pour Îles françaises de l'Amérique par la Compagnie des Indes
Pas de résultat pour Colonies Françaises de l'Amérique
Pas de résultat pour Rennes
Pas de résultat pour Bourgogne
Pas de résultat pour Le Havre
Pas de résultat pour Paris


[{'direction': 'Bayonne', 'carre_du_marche': 0.05187351022552995},
 {'direction': 'Bordeaux', 'carre_du_marche': 0.15313622333510415},
 {'direction': 'Caen', 'carre_du_marche': 0.1425434992682523},
 {'direction': 'Flandre', 'carre_du_marche': 0.07045997055524397},
 {'direction': 'La Rochelle', 'carre_du_marche': 0.2372572195962676},
 {'direction': 'Lyon', 'carre_du_marche': 0.1091605770171786},
 {'direction': 'Marseille', 'carre_du_marche': 0.07092719733421061},
 {'direction': 'Montpellier', 'carre_du_marche': 0.11633090857861035},
 {'direction': 'Nantes', 'carre_du_marche': 0.15998648838608395},
 {'direction': 'Rouen', 'carre_du_marche': 0.1397437786548302},
 {'direction': 'Amiens', 'carre_du_marche': 0.2996099650923989},
 {'direction': 'Auch', 'carre_du_marche': 0.27257857864021223},
 {'direction': 'Besançon', 'carre_du_marche': 0.11788428688554205},
 {'direction': 'Charleville', 'carre_du_marche': 0.15837474012943928},
 {'direction': 'Châlons', 'carre_du_marche': 0.08556529807816997

# Résultats pour les bureaux

In [147]:
bureaux_table = duckdb.query("""
    SELECT DISTINCT customs_region
    FROM toflit_flows
    WHERE customs_office IS NOT NULL
    AND product_revolutionempire IS NOT NULL
""")
bureaux = [tuple[0] for tuple in bureaux_table.fetchall()]

In [148]:
carre_du_marche_bureaux = []
for bureau in bureaux:
    try:
        carre = carre_du_marche(
            export_import='exports',
            year='1789',
            bureau_direction='customs_region',
            nom=bureau)
    except:
        print(f"Pas de résultat pour {bureau}")
    else:
        carre_du_marche_bureaux.append({
            'bureau': bureau,
            'carre_du_marche' : carre
        })

carre_du_marche_bureaux

Pas de résultat pour None


[{'bureau': 'Flandre', 'carre_du_marche': 0.07045997055524397},
 {'bureau': 'Amiens', 'carre_du_marche': 0.2996099650923989},
 {'bureau': 'Auch', 'carre_du_marche': 0.27257857864021223},
 {'bureau': 'Bayonne', 'carre_du_marche': 0.05187351022552995},
 {'bureau': 'Besançon', 'carre_du_marche': 0.11788428688554205},
 {'bureau': 'Bordeaux', 'carre_du_marche': 0.15313622333510415},
 {'bureau': 'Caen', 'carre_du_marche': 0.1425434992682523},
 {'bureau': 'Charleville', 'carre_du_marche': 0.15837474012943928},
 {'bureau': 'Châlons', 'carre_du_marche': 0.08556529807816997},
 {'bureau': 'Directions de terre', 'carre_du_marche': 0.5295238986839408},
 {'bureau': 'France', 'carre_du_marche': 0.05988757558038836},
 {'bureau': 'Grenoble', 'carre_du_marche': 0.09252546759991014},
 {'bureau': 'La Rochelle', 'carre_du_marche': 0.2372572195962676},
 {'bureau': 'Langres', 'carre_du_marche': 0.2687576638827042},
 {'bureau': 'Lorient', 'carre_du_marche': 0.1864742232674505},
 {'bureau': 'Lyon', 'carre_du_m

## Toute la France

In [ ]:
relation = duckdb.sql(f"""
    FROM base_relation
    WHERE customs_region IS NOT NULL
    """)

all_product_value = relation.sum('value').fetchone()[0]

aggreg = duckdb.sql(f"""
SELECT product_revolutionempire, SUM(value) AS sum_value
FROM relation
GROUP BY product_revolutionempire
""")

carre = duckdb.sql(f"""
SELECT  product_revolutionempire,
        sum_value, {all_product_value} AS total,
        power(sum_value/{all_product_value}, 2) AS carre_part_marche
FROM aggreg
""")

carre.sum('carre_part_marche').fetchone()[0]

0.06499554121933382